# Install


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import pandas as pd

import process_transactions
import requisition
import utils

load_dotenv("../user.env")

/var/folders/rp/f2ppsq3x6s33kb4f3qph_x1h0000gn/T/ipykernel_53046/110315734.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


True

# Get user data


In [3]:
user_data = await requisition.get_user_data()

2024-02-17 12:35:15.854 | INFO     | requisition:_get_requisition_id:29 - found 1 requisition_ids
2024-02-17 12:35:15.856 | DEBUG    | services.nordigen.account_api:load_requisition:214 - requisition already fetched
2024-02-17 12:35:15.858 | DEBUG    | services.nordigen.account_api:load_latest_account_data:197 - fetching account data


2024-02-17 12:35:16.763 | DEBUG    | services.nordigen.account_api:__init__:17 - Initialising API
2024-02-17 12:35:16.764 | DEBUG    | services.nordigen.account_api:__init__:17 - Initialising API
2024-02-17 12:35:16.765 | DEBUG    | services.nordigen.account_api:__init__:17 - Initialising API
2024-02-17 12:35:16.766 | DEBUG    | services.nordigen.account_api:get_transactions:22 - Getting transactions for 590300bd-3daf-4d5e-9274-7a3782261f7e
2024-02-17 12:35:16.787 | DEBUG    | services.nordigen.account_api:get_balances:30 - Getting balances for 590300bd-3daf-4d5e-9274-7a3782261f7e
2024-02-17 12:35:16.788 | DEBUG    | services.nordigen.account_api:get_transactions:22 - Getting transactions for d2ff77d0-6c80-4580-95a5-e3e87a098db9
2024-02-17 12:35:16.788 | DEBUG    | services.nordigen.account_api:get_balances:30 - Getting balances for d2ff77d0-6c80-4580-95a5-e3e87a098db9
2024-02-17 12:35:16.789 | DEBUG    | services.nordigen.account_api:get_transactions:22 - Getting transactions for e9e5

# Get updated transactions


In [4]:
await process_transactions.main()
user_data.transactions = utils.read_json(process_transactions.SAVE_PATH)

2024-02-17 12:35:22.873 | INFO     | requisition:_get_requisition_id:29 - found 1 requisition_ids
2024-02-17 12:35:22.874 | INFO     | services.nordigen.account_api:get_latest_data:226 - today's data exists


2024-02-17 12:35:22.958 | DEBUG    | process_transactions:combine_transactions:117 - joining historic and live transaction tables
2024-02-17 12:35:23.151 | INFO     | process_transactions:_enrich_transactions_dataframe:33 - Enriching dataframe
2024-02-17 12:35:23.156 | WARNING  | enrich.counterparty:get_counterparty:244 - unknown proprietaryBankTransactionCode: FASTER PAYMENT DEBIT REVERSAL 
2024-02-17 12:35:23.164 | INFO     | enrich.categories:categorize_transactions:35 - Adding transactions category


# Query


In [13]:
report_columns = [
    "booking_date",
    "remittance_information_unstructured",
    "proprietary_bank_transaction_code",
    "transaction_amount.amount",
    "status",
    "counterparty",
    "account_name",
    "account_type",
    "merchant_category_code",
    "merchant_category",
    "booking_year",
    "booking_month",
    "flow",
    "category",
]

In [22]:
class DataViewController:
    def __init__(self):
        """
        Initialize the DataViewController with transactions_df and accounts_df.
        """

    @staticmethod
    def pending(transactions_df: pd.DataFrame, account_name: str) -> float:
        """
        Calculate the pending amount for a given account name in the transactions DataFrame.

        :param transactions_df: A DataFrame containing transactions data.
        :param account_name: The name of the account for which the pending amount is to be calculated.
        :return: The pending amount for the given account name.
        """
        df = copy.deepcopy(transactions_df)
        df = df.loc[
            (transactions_df["account_name"] == account_name)
            & (df["status"] == "pending")
        ]
        return round(df["transactionAmount.amount"].sum(), 2)

    def account_balances(self, accounts_df: pd.DataFrame) -> dict:
        """
        Get the account balances DataFrame with columns: institution_name, account_name, Available, Pending, and Booked.

        :return: A DataFrame containing account balances.
        """

        col_mapping = {
            "balanceAmount.interimAvailable": "Available",
            "balanceAmount.interimBooked": "Booked",
        }
        cols = ["account_name", "Available", "Pending", "Booked"]

        df = copy.deepcopy(accounts_df)
        df = df.rename(columns=col_mapping)
        df["Pending"] = df["Available"] - df["Booked"]
        df = df.loc[df["account_type"] == "CACC"].dropna(axis=1)[cols]
        return df.to_dict(orient="records")

    def credit_card_balances(
        self,
        transactions_df: pd.DataFrame,
        accounts_df: pd.DataFrame,
    ) -> dict:
        """
        Get the credit card balances DataFrame with columns: institution_name, account_name, Balance, Pending, and Available.

        :return: A DataFrame containing credit card balances.
        """
        col_mapping = {
            "balanceAmount.forwardAvailable": "Available",
            "balanceAmount.openingCleared": "Balance",
        }
        cols = ["account_name", "Balance", "Pending", "Available"]
        df = copy.deepcopy(accounts_df)

        pending = []
        for account_name in accounts_df["account_name"]:
            pending.append(self.pending(transactions_df, account_name))
        df["Pending"] = pending
        df = df.rename(columns=col_mapping)
        df = df.loc[df["account_type"] == "CARD"].dropna(axis=1)[cols]
        return df.to_dict(orient="records")


dvc = DataViewController()

In [20]:
transactions = pd.DataFrame(user_data.transactions).dropna(axis=1).drop_duplicates()